#### Task 1 - Data Cleaning

Provided is code that reads a .csv file into a pandas dataframe. The dataframe includes some inconsistent country names, and your task ist to fill out the code snippet marked below which contains a dictionary mapping the inconsistent country names to the naming scheme most of the dataframe uses.  

In [1]:
import pandas as pd

In [2]:
def load_gdp_data():
    #load dataset
    gdp = pd.read_csv('data/GDPPC_data.csv',na_values=['..'])
    #drop unwanted columns
    gdp.drop(['Indicator Name','Indicator Code','Country Code' ], inplace=True, axis=1)
    #reshape dataframe
    gdp.set_index(['Country Name'], inplace=True)
    gdp=gdp.stack(dropna=False).to_frame()
    gdp.reset_index(drop=False, inplace=True)
    #rename columns
    gdp=gdp.rename(columns={'Country Name': 'country', 'level_1': 'year', 0: 'GDP'})
    #set datatype for year, GDP
    gdp['year']=gdp['year'].astype(int)
    gdp['GDP']=gdp['GDP'].astype(float)
    #only use data from 2000 onwards
    gdp = gdp[gdp.year >= 2000]
    #set index
    gdp=gdp.set_index(['country', 'year'])
    return gdp

gdp_df=load_gdp_data()

In [3]:
def load_hdi_data():
    #load dataset
    hdi = pd.read_csv('data/HDI.csv',na_values=['..'])
    #drop unwanted columns
    hdi.drop(['HDI Rank (2018)'], inplace=True, axis=1)
    #reshape dataframe
    hdi.set_index(['Country'], inplace=True)
    hdi=hdi.stack(dropna=False).to_frame()
    hdi.reset_index(drop=False, inplace=True)
    #rename columns
    hdi=hdi.rename(columns={'Country': 'country', 'level_1': 'year', 0: 'HDI'})
    #set datatype for year
    hdi['year']=hdi['year'].astype(int)
    hdi['HDI']=hdi['HDI'].astype(float)
    #only use data from 2000 onwards
    hdi = hdi[hdi.year >= 2000]
    #set index
    hdi=hdi.set_index(['country', 'year'])
    return hdi

hdi_df=load_hdi_data()

In [4]:
def load_oecd_data():
    oecd = pd.read_csv('data/oecd.csv',na_values=['..'])
    oecd = oecd.set_index(['country', 'year'])
    return oecd

oecd_df = load_oecd_data()

In [5]:
def country_check(countries, gdp_df, hdi_df):
    #get country name list series
    hdi_c = hdi_df.index.unique(level=0).to_series()
    gdp_c = gdp_df.index.unique(level=0).to_series()
    
    #create dataframe for the results
    check=pd.DataFrame(columns=['country','hdi','gdp'])
    #iterate through oecd_df source or destination country names and check presence of country name in other dfs
    for index,row in countries.iterrows():
        check=check.append({'country': row.country,
                            'hdi':row.isin(hdi_c).values[0],
                            'gdp':row.isin(gdp_c).values[0]}, ignore_index=True)
        
    #add only problematic country rows to the result df
    check=check.loc[(check['hdi'] == False) | (check['gdp'] == False)]
    return check

oecd_s = pd.DataFrame(oecd_df.index.unique(level=0))
oecd_d = pd.DataFrame(oecd_df.destination.unique(), columns={'country'})

pd.concat([country_check(oecd_d, gdp_df, hdi_df), country_check(oecd_s, gdp_df, hdi_df)])

,country,hdi,gdp
5,Czech Republic,False,True
18,Korea,False,False
27,Slovak Republic,False,True
11,Bahamas,True,False
20,Bolivia,False,True
31,Cape Verde,False,False
38,Congo,True,False
43,Czech Republic,False,True
44,Côte d'Ivoire,True,False
45,Democratic Republic of the Congo,False,False


sideline - link to subplot 1

In [8]:
country_dict = {
    'Bahamas, The': 'Bahamas',
}

In [9]:
def country_correction(df):
    return df.rename(index=country_dict, level=0)

#call country_correction() to rename country names
hdi_df = country_correction(hdi_df)
gdp_df = country_correction(gdp_df)

#check for inconsistencies again
oecd_s = pd.DataFrame(oecd_df.index.unique(level=0))
oecd_d = pd.DataFrame(oecd_df.destination.unique(), columns={'country'})

oecd_check = pd.concat([country_check(oecd_d, gdp_df, hdi_df), country_check(oecd_s, gdp_df, hdi_df)])

if oecd_check.empty:
    print("No more inconsistent country names!")
else:
    print("Some inconsistencies remain:")
    display(oecd_check)

Some inconsistencies remain:


,country,hdi,gdp
5,Czech Republic,False,True
18,Korea,False,False
27,Slovak Republic,False,True
11,Bahamas,True,False
20,Bolivia,False,True
31,Cape Verde,False,False
38,Congo,True,False
43,Czech Republic,False,True
44,Côte d'Ivoire,True,False
45,Democratic Republic of the Congo,False,False


This task illustrates the utility of subplots as a navigation aid. In the cells below are the unqiue country names from the hdi and gdp dataframes.

In [6]:
hdi_df.reset_index().country.unique()[:, None]

array([['Afghanistan'],
       ['Albania'],
       ['Algeria'],
       ['Andorra'],
       ['Angola'],
       ['Antigua and Barbuda'],
       ['Argentina'],
       ['Armenia'],
       ['Australia'],
       ['Austria'],
       ['Azerbaijan'],
       ['Bahamas'],
       ['Bahrain'],
       ['Bangladesh'],
       ['Barbados'],
       ['Belarus'],
       ['Belgium'],
       ['Belize'],
       ['Benin'],
       ['Bhutan'],
       ['Bolivia (Plurinational State of)'],
       ['Bosnia and Herzegovina'],
       ['Botswana'],
       ['Brazil'],
       ['Brunei Darussalam'],
       ['Bulgaria'],
       ['Burkina Faso'],
       ['Burundi'],
       ['Cabo Verde'],
       ['Cambodia'],
       ['Cameroon'],
       ['Canada'],
       ['Central African Republic'],
       ['Chad'],
       ['Chile'],
       ['China'],
       ['Colombia'],
       ['Comoros'],
       ['Congo'],
       ['Congo (Democratic Republic of the)'],
       ['Costa Rica'],
       ['Croatia'],
       ['Cuba'],
       ['Cyprus'],
   

In [7]:
gdp_df.reset_index().country.unique()[:, None]

array([['Aruba'],
       ['Afghanistan'],
       ['Angola'],
       ['Albania'],
       ['Andorra'],
       ['Arab World'],
       ['United Arab Emirates'],
       ['Argentina'],
       ['Armenia'],
       ['American Samoa'],
       ['Antigua and Barbuda'],
       ['Australia'],
       ['Austria'],
       ['Azerbaijan'],
       ['Burundi'],
       ['Belgium'],
       ['Benin'],
       ['Burkina Faso'],
       ['Bangladesh'],
       ['Bulgaria'],
       ['Bahrain'],
       ['Bahamas, The'],
       ['Bosnia and Herzegovina'],
       ['Belarus'],
       ['Belize'],
       ['Bermuda'],
       ['Bolivia'],
       ['Brazil'],
       ['Barbados'],
       ['Brunei Darussalam'],
       ['Bhutan'],
       ['Botswana'],
       ['Central African Republic'],
       ['Canada'],
       ['Central Europe and the Baltics'],
       ['Switzerland'],
       ['Channel Islands'],
       ['Chile'],
       ['China'],
       ["Cote d'Ivoire"],
       ['Cameroon'],
       ['Congo, Dem. Rep.'],
       ['Congo, Re